rough mayorga

In [1]:
import math
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression, RFE

In [2]:
## Trusty Modules

from Acquire import get_texas_data
from Prepare import prepare_tex, create_features, split_data
import m_explore as e

ModuleNotFoundError: No module named 'acquire'

# Acquire

In [ ]:
df = get_texas_data()

In [ ]:
df.info()

# Prepare

In [ ]:
df = prepare_tex(df)

In [ ]:
df.info()

In [ ]:
df.head()

# Explore

In [ ]:
sample_df = df.sample(10000)

In [ ]:
sample_df.columns

In [ ]:
df.race.value_counts()

In [ ]:
df = create_features(df)

In [ ]:
df.head()

In [ ]:
df.is_BIPOC.value_counts()

## Split the data

In [ ]:
train, validate, test = split_data(df)

In [ ]:
sns.relplot(x='tenure_years', y='annual_salary', data=train)